In [8]:
import pandas as pd
import numpy as np
import re
from string import punctuation
%pip install matplotlib tqdm unidecoce
import matplotlib.pyplot as plt
from tqdm import tqdm
from unidecode import unidecode
%pip install scikit-learn scipy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

from scipy.stats import pearsonr
from scipy.stats import spearmanr

%pip install -U nltk
import nltk
nltk.download("punkt")
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
%pip install spacy
!spacy download en_core_web_sm
import spacy
spacy_nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
%pip install gensim
from gensim.models import Word2Vec
import gensim.downloader as gensim_api

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement unidecoce (from versions: none)
ERROR: No matching distribution found for unidecoce


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Note: you may need to restart the kernel to use updated packages.
                                              0.0/12.8 MB ? eta -:--:--
     -                                        0.5/12.8 MB 10.0 MB/s eta 0:00:02
     ---                                      1.3/12.8 MB 16.0 MB/s eta 0:00:01
     -------                                  2.3/12.8 MB 16.0 MB/s eta 0:00:01
     ----------                               3.5/12.8 MB 18.6 MB/s eta 0:00:01
     --------------                           4.8/12.8 MB 21.9 MB/s eta 0:00:01
     -----------------                        5.5/12.8 MB 19.6 MB/s eta 0:00:01
     -----------------------                  7.6/12.8 MB 23.1 MB/s eta 0:00:01
     ------------------------------          10.1/12.8 MB 28.0 MB/s eta 0:00:01
     -------------------------------         10.4/12.8 MB 26.2 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 31.2 MB/s eta 0:00:01
     --------------------------------------  12.8/12.8 MB 31.2

In [3]:
def on_bad_line(values):
    return values[:7]

columns_mapping = {
    0: 'genre',
    1: 'filename',
    2: 'year',
    3: 'index',
    4: 'score',
    5: 'sentence1',
    6: 'sentence2'
}

# quotings 3 = csv.QUOTE_NONE
train_df = pd.read_csv('Dataset/sts-train.csv', sep="\t", on_bad_lines=on_bad_line, engine='python', header=None, encoding='utf-8', quoting=3).rename(columns=columns_mapping)
print(f'shape of the Dataframe {train_df.shape}')
train_df.head(10)

shape of the Dataframe (5749, 7)


,genre,filename,year,index,score,sentence1,sentence2
0,main-captions,MSRvid,2012test,1,5.00,A plane is taking off.,An air plane is taking off.
1,main-captions,MSRvid,2012test,4,3.80,A man is playing a large flute.,A man is playing a flute.
2,main-captions,MSRvid,2012test,5,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,main-captions,MSRvid,2012test,6,2.60,Three men are playing chess.,Two men are playing chess.
4,main-captions,MSRvid,2012test,9,4.25,A man is playing the cello.,A man seated is playing the cello.
5,main-captions,MSRvid,2012test,11,4.25,Some men are fighting.,Two men are fighting.
6,main-captions,MSRvid,2012test,12,0.50,A man is smoking.,A man is skating.
7,main-captions,MSRvid,2012test,13,1.60,The man is playing the piano.,The man is playing the guitar.
8,main-captions,MSRvid,2012test,14,2.20,A man is playing on a guitar and singing.,A woman is playing an acoustic guitar and sing...
9,main-captions,MSRvid,2012test,16,5.00,A person is throwing a cat on to the ceiling.,A person throws a cat on the ceiling.


In [4]:
dev_df = pd.read_csv('Dataset/sts-dev.csv', sep="\t", on_bad_lines=on_bad_line, engine='python', header=None, encoding='utf-8', quoting=3).rename(columns=columns_mapping)
print(f'shape of the Dataframe {dev_df.shape}')

test_df = pd.read_csv('Dataset/sts-test.csv', sep="\t", on_bad_lines=on_bad_line, engine='python', header=None, encoding='utf-8', quoting=3).rename(columns=columns_mapping)
print(f'shape of the Dataframe {test_df.shape}')

shape of the Dataframe (1500, 7)
shape of the Dataframe (1379, 7)


In [5]:
sentences1 = train_df["sentence1"]
sentences2 = train_df["sentence2"]
sentences_list = np.unique(np.concatenate((np.ravel(sentences1.values),np.ravel(sentences2.values))))

dev_sentences1 = dev_df["sentence1"]
dev_sentences2 = dev_df["sentence2"]
dev_sentences_list = np.unique(np.concatenate((np.ravel(dev_sentences1.values),np.ravel(dev_sentences2.values))))

print(f"Total number of train sentences: {len(sentences_list)}")
print(f"Total number of dev sentences: {len(dev_sentences_list)}")

Total number of train sentences: 10566
Total number of dev sentences: 2910


In [10]:
def remove_special_chars(text):
    regex = '[' + punctuation + ']'
    return re.sub(regex,'',unidecode(text)).lower()

def remove_stop_words(text):
    stop_words = stopwords.words('english')
    return ' '.join([word for word in text.split() if word not in stop_words])

def lemmatize(text):
    parsed_sentence = spacy_nlp(str(text))
    lemmatized_sentence = ' '.join([word.lemma_ for word in parsed_sentence])
    return lemmatized_sentence

def substitute_number(sentence):
    return re.sub(r'\d+', ' number ', sentence)

#TODO mostrare differenze tra originale e lemmatizzazione

no_punct_sentences_list = np.array([remove_special_chars(string) for string in sentences_list])
clean_sentences_list = np.array([remove_stop_words(string) for string in no_punct_sentences_list])
lemmatized_sentences_list = np.array([remove_stop_words(lemmatize(remove_special_chars(sentence)).lower()) for sentence in clean_sentences_list])

no_punct_dev_sentences_list = np.array([remove_special_chars(string) for string in dev_sentences_list])
clean_dev_sentences_list = np.array([remove_stop_words(string) for string in no_punct_dev_sentences_list])
lemmatized_dev_sentences_list = np.array([remove_stop_words(lemmatize(remove_special_chars(sentence)).lower()) for sentence in clean_dev_sentences_list])



print(sentences_list[:5])
print(clean_sentences_list[:5])
print(lemmatized_sentences_list[:5])

[' - Income, top rate:  percent.' ' Foxe was mostly frozen.'
 " billion . Everyone in the world at Mexico's current prosperity level."
 ' warmed.C from the year prior.'
 '"Americans don\'t cut and run, we have to see this misadventure through," she said.']
['income top rate percent' 'foxe mostly frozen'
 'billion everyone world mexicos current prosperity level'
 'warmedc year prior' 'americans dont cut run see misadventure said']
['income top rate percent' 'foxe mostly frozen'
 'billion everyone world mexicos current prosperity level'
 'warmedc year prior' 'americans cut run see misadventure say']


In [11]:
array_len = np.vectorize(len)(sentences_list)
mean_len = np.mean(array_len)
print(f"Mean length of sentences in characters: {mean_len:.2f}")

Mean length of sentences in characters: 59.37


In [28]:
clean_dataset = pd.DataFrame({
    'sentence1': train_df['sentence1'].map(remove_special_chars).map(remove_stop_words),
    'sentence2': train_df['sentence2'].map(remove_special_chars).map(remove_stop_words),
    'score': train_df['score'] / 5
})

clean_test_dataset = pd.DataFrame({
    'sentence1': test_df['sentence1'].map(remove_special_chars).map(remove_stop_words),
    'sentence2': test_df['sentence2'].map(remove_special_chars).map(remove_stop_words),
    'score': test_df['score'] / 5
})

no_number_dataset = pd.DataFrame({
    'sentence1': clean_dataset['sentence1'].map(substitute_number),
    'sentence2': clean_dataset['sentence2'].map(substitute_number),
    'score': clean_dataset['score']
})

no_number_test_dataset = pd.DataFrame({
    'sentence1': clean_test_dataset['sentence1'].map(substitute_number),
    'sentence2': clean_test_dataset['sentence2'].map(substitute_number),
    'score': clean_test_dataset['score']
})

lemmatized_dataset = pd.DataFrame({
    'sentence1': clean_dataset['sentence1'].map(lemmatize),
    'sentence2': clean_dataset['sentence2'].map(lemmatize),
    'score': clean_dataset['score']
})

lemmatized_test_dataset = pd.DataFrame({
    'sentence1': clean_test_dataset['sentence1'].map(lemmatize),
    'sentence2': clean_test_dataset['sentence2'].map(lemmatize),
    'score': clean_test_dataset['score']
})

In [37]:
import os
from sentence_transformers import util

wikipedia_filepath = 'simplewiki-2020-11-01.jsonl.gz'
if not os.path.exists(wikipedia_filepath):
    util.http_get('http://sbert.net/datasets/simplewiki-2020-11-01.jsonl.gz', wikipedia_filepath)
import json
import gzip

passages = []
# Open the file with the dump of Simple Wikipedia
with gzip.open(wikipedia_filepath, 'rt', encoding='utf8') as f:
    # Iterate over the lines
    for line in f:
        # Parse the document using JSON
        data = json.loads(line.strip())
        # Add all paragraphs
        #passages.extend(data['paragraphs'])
        # Only add the first paragraph
        passages.append(data['paragraphs'][0])

print(f"Retreived {len(passages)} passages")

100%|██████████| 50.2M/50.2M [00:01<00:00, 25.1MB/s]


Retreived 169597 passages


In [39]:
no_punct_passages = np.array([remove_special_chars(string) for string in passages])
clean_passages = np.array([remove_stop_words(string) for string in no_punct_passages])
lemmatized_passages = np.array([remove_stop_words(lemmatize(remove_special_chars(sentence)).lower()) for sentence in clean_passages])

In [40]:
# saving no punct, clean and lemmatized passages in txt files
np.savetxt("Dataset/no_punct_passages.txt", no_punct_passages, fmt="%s")
np.savetxt("Dataset/clean_passages.txt", clean_passages, fmt="%s")
np.savetxt("Dataset/lemmatized_passages.txt", lemmatized_passages, fmt="%s")

In [41]:
# tokenized_sentences = [word_tokenize(w) for w in np.concatenate((lemmatized_sentences_list, lemmatized_dev_sentences_list))]
# tokenized_sentences = [word_tokenize(w) for w in lemmatized_sentences_list]
tokenized_sentences = [word_tokenize(w) for w in lemmatized_passages]

In [42]:
tokenized_sentences

[['te',
  'cassidy',
  'july',
  '31',
  '1932',
  'january',
  '16',
  '1979',
  'american',
  'actor',
  'well',
  'know',
  'role',
  'lurch',
  'thing',
  'addam',
  'family'],
 ['aileen',
  'carol',
  'wuornos',
  'pralle',
  'bear',
  'aileen',
  'carol',
  'pittman',
  'february',
  '29',
  '1956',
  'october',
  '9',
  '2002',
  'american',
  'serial',
  'killer',
  'bear',
  'rochester',
  'michigan',
  'confess',
  'kill',
  'six',
  'man',
  'florida',
  'execute',
  'florida',
  'state',
  'prison',
  'lethal',
  'injection',
  'murder',
  'wuorno',
  'say',
  'man',
  'kill',
  'rape',
  'try',
  'rape',
  'work',
  'prostitute'],
 ['crater',
  'round',
  'dent',
  'planet',
  'usually',
  'shape',
  'like',
  'circle',
  'oval',
  'usually',
  'make',
  'something',
  'like',
  'meteor',
  'hit',
  'surface',
  'planet',
  'underground',
  'activity',
  'volcano',
  'explosion',
  'also',
  'cause',
  'likely'],
 ['store', 'several', 'meaning'],
 ['chinese',
  'new',
  'y

In [43]:
model = Word2Vec(tokenized_sentences, vector_size=50, min_count=4, window=5, negative=10, workers=4, sg=0, cbow_mean=1, epochs=150, seed=42)
print("vocabular size:", len(model.wv.key_to_index))

#print most similar words to
for item in ['car', 'boy', 'man', 'girl', 'woman', 'obama', 'florida', 'cat', 'dog', 'blue', 'kill', 'bomb', 'piano', 'pizza', 'italy']:
    print(item, '-->', [w for w, s in model.wv.most_similar(item)])

vocabular size: 49226
car --> ['truck', 'vehicle', 'motor', 'automobile', 'roadster', 'toyota', 'rearwheel', 'offroad', 'allwheel', 'fullsize']
boy --> ['girl', 'pretty', 'young', 'nsync', 'man', 'teenage', 'dream', 'freak', 'little', 'friend']
man --> ['woman', 'disguise', 'naked', 'boy', 'love', 'young', 'lucky', 'invisible', 'birdcage', 'whoever']
girl --> ['boy', 'teenager', 'teenage', 'child', 'teen', 'kid', 'pretty', 'crazy', 'friend', 'man']
woman --> ['man', 'person', 'child', 'sexuality', 'disguise', 'homosexual', 'female', 'womans', 'overweight', 'abusive']
obama --> ['barack', 'obamas', 'trump', 'nixon', 'approval', 'clinton', 'presidential', 'rousseff', 'reagan', 'truman']
florida --> ['texas', 'oklahoma', 'alabama', 'california', 'miami', 'georgia', 'arizona', 'carolina', 'jacksonville', 'illinois']
cat --> ['rabbit', 'dog', 'pet', 'ferret', 'raccoon', 'felidae', 'anthropomorphic', 'hound', 'bicolor', 'naughty']
dog --> ['cat', 'breed', 'laika', 'badger', 'stoat', 'naughty

In [15]:
def vectorize(sentences, vocab, vectorizer, vector_size=50):
    # create a matrix with rows equal to the number of tweets and columns equal to the embedding size
    vectors = np.zeros((len(sentences), vector_size))

    # for each sentence in the dataset compute the embedding 
    for i, sentence in enumerate(sentences):
        tokens = word_tokenize(sentence)
        embeddings = [vectorizer(token) for token in tokens if token in vocab] # embedding per ogni token
        if (len(embeddings) > 0): # unisce tutti gli embedding in base a useSum
            vectors[i] = np.mean(embeddings, axis=0)
    return vectors

In [21]:
# modified version of vectorize that weightes the embeddings by the tf-idf score of the word
def vectorize_tfidf(sentences, vocab, vectorizer, tfidf_matrix, useSum=False, vector_size=50):
    # create a matrix with rows equal to the number of tweets and columns equal to the embedding size
    vectors = np.zeros((len(sentences), vector_size))

    # for each sentence in the dataset compute the embedding 
    for i, sentence in enumerate(sentences):
        tokens = word_tokenize(sentence)
        embeddings = [vectorizer(token) * tfidf_matrix[i, vocab.index(token)] for token in tokens if token in vocab] # embedding per ogni token
        if (len(embeddings) > 0): # unisce tutti gli embedding in base a useSum
            if (useSum): 
                vectors[i] = sum(embeddings)
            else:
                vectors[i] = np.mean(embeddings, axis=0)
    return vectors

In [44]:
predicted_scores = []
dataset = lemmatized_test_dataset
for idx, row in dataset.iterrows():
    sentences = [row['sentence1'], row['sentence2']]
    vectors = vectorize(sentences, model.wv, model.wv.get_vector)
    similarity = cosine_similarity(vectors)[0][1]
    similarity = np.clip(similarity, -1, 1)
    predicted_scores.append(similarity)

pearson = pearsonr(dataset["score"].values, predicted_scores)
spearman = spearmanr(dataset["score"].values, predicted_scores)
print("Pearson:", pearson)
print("Spearman:", spearman)

Pearson: PearsonRResult(statistic=0.5368970773058587, pvalue=8.426304078384008e-104)
Spearman: SignificanceResult(statistic=0.5801921571130488, pvalue=7.083919901233745e-125)


WMD algorithm

In [25]:
!pip install pot

                                              0.0/228.3 kB ? eta -:--:--
     -------------------------------------- 228.3/228.3 kB 4.6 MB/s eta 0:00:00


In [47]:
predicted_scores = []
for idx, row in test_df.iterrows():
    similarity = -model.wv.wmdistance(row['sentence1'], row['sentence2'])
    predicted_scores.append(similarity)

pearson = pearsonr(test_df["score"].values, predicted_scores)
spearman = spearmanr(test_df["score"].values, predicted_scores)
print("Pearson:", pearson)
print("Spearman:", spearman)

Pearson: PearsonRResult(statistic=0.4701716206129475, pvalue=9.099637411986036e-77)
Spearman: SignificanceResult(statistic=0.49343687571941025, pvalue=1.607178606582556e-85)
